# 安装插件

In [58]:
!pip install yfinance pandas openpyxl

# 第一题-第一小题

数据描述与来源:
----------------------------------------------------------------------------------------------------
数据内容:
选定的一系列交易所交易基金 (ETF) 的历史日度行情数据。
对于每个ETF，获取的数据通常包含以下字段：
  - Date: 交易日期
  - Open: 当日开盘价
  - High: 当日最高价
  - Low: 当日最低价
  - Close: 当日收盘价
  - Adj Close: 调整后收盘价 (已针对股息和股票分割进行调整，通常用于计算回报率)
  - Volume: 当日成交量

计算的统计指标:
  - 每日回报率 (Daily Returns): 根据“调整后收盘价 (Adj Close)”计算得出，公式为 (今日价格 / 昨日价格) - 1。
  - 平均每日回报率 (Average Daily Return): 每个ETF在指定时间段内每日回报率的算术平均值。
  - 每日回报率标准差 (Standard Deviation of Daily Returns):衡量每个ETF每日回报率波动性或风险的指标。
  - 相关性矩阵 (Correlation Matrix): 显示所选ETF每日回报率之间的两两相关系数。
    - 系数接近 +1 表示强正相关 (价格倾向于同向变动)。
    - 系数接近 -1 表示强负相关 (价格倾向于反向变动)。
    - 系数接近 0 表示线性相关性较弱或无线性相关。

数据来源:
所有金融数据均通过 `yfinance` Python 库从 Yahoo Finance (雅虎财经) 获取。
Yahoo Finance 是一个广泛使用的金融市场数据、新闻、分析和工具提供商。
`yfinance` 库为访问这些公开可用的数据提供了便捷的接口。

ETF选择:
脚本中选择的行业及代码如下，旨在代表美国股市的不同行业板块：

- "医疗保健 (Health Care)": "XLV",
- "农林牧渔 (Agriculture)": "DBA",  # Invesco DB Agriculture Fund
- "采矿业 (Mining)": "XME",  # SPDR S&P Metals & Mining ETF
- "制造业 (Industrials/Manufacturing)": "XLI",  # Industrial Select Sector SPDR Fund (覆盖广泛的制造业和工业)
- "水电煤气 (Utilities)": "XLU",  # Utilities Select Sector SPDR Fund
- "房地产 (Real Estate)": "XLRE",
- "批发和零售业 (Retail)": "XRT",  # SPDR S&P Retail ETF
- "交通运输 (Transportation)": "IYT",  # iShares U.S. Transportation ETF
- "住宿和餐饮业 (Consumer Discretionary/Travel)": "PEJ", # Invesco - Leisure and Entertainment ETF (更侧重休闲娱乐，间接包含部分)
- "金融业 (Financials)": "XLF"  # Financial Select Sector SPDR Fund


数据时间范围:
指定数据的开始(20150101)和结束日期(20241231)。

----------------------------------------------------------------------------------------------------

In [59]:
# 导入所需库
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

def fetch_analyze_and_save_industry_data(start_date_str, end_date_str, output_filename="industry_data.xlsx"):
    """
    获取指定日期范围内10个行业的代表性ETF数据，进行基本分析，并保存到Excel文件中。

    参数:
    start_date_str (str): 开始日期，格式 "YYYY-MM-DD"
    end_date_str (str): 结束日期，格式 "YYYY-MM-DD"
    output_filename (str): 输出的Excel文件名
    """

    # 定义10个行业的代表性ETF代码及其名称
    industry_etfs = {
    "医疗保健 (Health Care)": "XLV",
    "农林牧渔 (Agriculture)": "DBA",  # Invesco DB Agriculture Fund
    "采矿业 (Mining)": "XME",  # SPDR S&P Metals & Mining ETF
    "制造业 (Industrials/Manufacturing)": "XLI",  # Industrial Select Sector SPDR Fund (覆盖广泛的制造业和工业)
    "水电煤气 (Utilities)": "XLU",  # Utilities Select Sector SPDR Fund
    "房地产 (Real Estate)": "XLRE",
    "批发和零售业 (Retail)": "XRT",  # SPDR S&P Retail ETF
    "交通运输 (Transportation)": "IYT",  # iShares U.S. Transportation ETF
    "住宿和餐饮业 (Consumer Discretionary/Travel)": "PEJ", # Invesco Leisure and Entertainment ETF (更侧重休闲娱乐，间接包含部分)
    "金融业 (Financials)": "XLF"  # Financial Select Sector SPDR Fund
    }

    # 验证日期格式
    try:
        datetime.strptime(start_date_str, "%Y-%m-%d")
        datetime.strptime(end_date_str, "%Y-%m-%d")
    except ValueError:
        print("错误：日期格式不正确。请使用 YYYY-MM-DD 格式。")
        return

    all_adj_closes = pd.DataFrame()
    etf_returns_data = {} # 用于存储每个ETF的回报率数据以供分析


    # 创建一个Excel写入器
    try:
        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
            print(f"开始获取数据并写入到 {output_filename}...")

            for industry_name, ticker_symbol in industry_etfs.items():
                print(f"\n正在获取 {industry_name} ({ticker_symbol}) 的数据...")
                try:
                    # 下载ETF历史数据
                    # yfinance会自动调整未来的结束日期到当前可用数据
                    data = yf.download(ticker_symbol, start=start_date_str, end=end_date_str, progress=False)

                    if data.empty:
                        print(f"未能获取到 {ticker_symbol} 的数据。可能该ETF在该时间段内无数据或代码有误。")
                        # 创建一个空的工作表，以表明尝试过
                        empty_df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
                        empty_df.to_excel(writer, sheet_name=ticker_symbol, index_label="Date")
                        etf_returns_data[ticker_symbol] = pd.Series(dtype=float) # 添加空Series
                    else:
                        # 将数据写入到Excel的不同工作表中
                        # 使用ETF代码作为工作表名称，因为它通常比行业名称更简洁且唯一
                        data.to_excel(writer, sheet_name=ticker_symbol, index_label="Date")
                        print(f"成功获取并已将 {ticker_symbol} ({industry_name}) 的数据写入到工作表 '{ticker_symbol}'。")
                        print(f"获取数据条数: {len(data)}")
                        print(f"数据起始日期: {data.index.min().strftime('%Y-%m-%d')}")
                        print(f"数据结束日期: {data.index.max().strftime('%Y-%m-%d')}")

                        # 计算每日回报率
                        # 使用 .copy() 避免 SettingWithCopyWarning
                        adj_close_series = data['Close'].copy()
                        daily_returns = adj_close_series.pct_change().dropna()
                        etf_returns_data[ticker_symbol] = daily_returns

                        # 将调整后收盘价添加到汇总DataFrame，用于计算相关性
                        # 重命名列以包含ETF代码，避免列名冲突
                        all_adj_closes[ticker_symbol] = adj_close_series

                except Exception as e:
                    print(f"获取 {ticker_symbol} ({industry_name}) 数据时发生错误: {e}")
                    # 创建一个空的工作表，并记录错误信息
                    error_df = pd.DataFrame({'Error': [str(e)]})
                    error_df.to_excel(writer, sheet_name=f"{ticker_symbol}_Error", index=False)

            print(f"\n所有数据处理完毕。Excel文件 '{output_filename}' 已保存。")

    except Exception as e:
        print(f"创建或写入Excel文件时发生错误: {e}")

    # --- 数据分析部分 ---
    print("\n--- 数据分析结果 ---")

    # 1. 计算平均每日回报率和标准差
    print("\n1. 平均每日回报率和标准差 (基于调整后收盘价):")
    analysis_results = []
    for ticker_symbol, returns in etf_returns_data.items():
        if not returns.empty:
            avg_return = returns.mean()
            std_dev = returns.std()
            analysis_results.append({
                "ETF": ticker_symbol,
                "行业": [name for name, tick in industry_etfs.items() if tick == ticker_symbol][0],
                "平均每日回报率": avg_return,
                "每日回报率标准差": std_dev,
                "年化平均回报率 (估算)": avg_return * 252, # 假设一年252个交易日
                "年化标准差 (估算)": std_dev * np.sqrt(252)
            })
        else:
            analysis_results.append({
                "ETF": ticker_symbol,
                "行业": [name for name, tick in industry_etfs.items() if tick == ticker_symbol][0],
                "平均每日回报率": np.nan,
                "每日回报率标准差": np.nan,
                "年化平均回报率 (估算)": np.nan,
                "年化标准差 (估算)": np.nan
            })
            print(f"  {ticker_symbol}: 无有效回报率数据进行分析。")

    results_df = pd.DataFrame(analysis_results)
    if not results_df.empty:
        print(results_df.to_string(index=False))


    # 2. 计算回报率的相关性矩阵
    # 首先，整合所有ETF的每日回报率到一个DataFrame
    # 确保所有回报率数据对齐日期索引
    if all_adj_closes.empty or all_adj_closes.shape[1] < 2:
        print("\n2. 相关性矩阵:")
        print("  未能收集到足够ETF的调整后收盘价数据来计算相关性矩阵。")
    else:
        returns_df_for_corr = all_adj_closes.pct_change().dropna()

        if returns_df_for_corr.shape[1] < 2 :
             print("\n2. 相关性矩阵:")
             print("  计算回报率后，有效ETF数量不足2个，无法计算相关性矩阵。")
        elif returns_df_for_corr.empty:
            print("\n2. 相关性矩阵:")
            print("  计算得到的回报率数据为空，无法计算相关性矩阵。")
        else:
            correlation_matrix = returns_df_for_corr.corr()
            print("\n2. 每日回报率相关性矩阵:")
            # 为了更好的可读性，格式化输出
            pd.set_option('display.width', None) # 确保打印完整的宽度
            pd.set_option('display.max_columns', None) # 确保打印所有列
            print(correlation_matrix.to_string(float_format="%.3f")) # 格式化浮点数为3位小数
            pd.reset_option('display.width')
            pd.reset_option('display.max_columns')




if __name__ == "__main__":
    # 设置获取数据的日期范围
    start_date_input = "2015-01-01"
    # 注意：如果end_date_input是未来日期，yfinance会自动获取到当前最新的可用数据。
    # 为了演示，如果当前日期早于2024年底，数据会截至到当前。
    # 如果当前日期晚于2024年底，那么2024-12-31这个日期是有效的历史日期。
    current_year = datetime.now().year
    if current_year < 2024:
        print(f"提示: 由于当前年份 ({current_year}) 早于请求的结束年份 (2024)，数据将获取至当前最新可用日期。")

    end_date_input = "2024-12-31"

    # 设置输出的Excel文件名
    excel_file_name = "行业ETF历史数据及分析_2015_至_2024.xlsx"

    # 调用函数执行数据获取、分析和保存
    fetch_analyze_and_save_industry_data(start_date_input, end_date_input, excel_file_name)

    # 提示用户如何查看数据
    print(f"\n您可以打开 '{excel_file_name}' 文件查看下载的原始价格数据。")
    print("每个行业ETF的原始价格数据存储在以其股票代码命名的不同工作表中。")
    print("统计分析结果（平均回报率、标准差、相关性矩阵）已打印在控制台输出中。")



开始获取数据并写入到 行业ETF历史数据及分析_2015_至_2024.xlsx...

正在获取 医疗保健 (Health Care) (XLV) 的数据...
成功获取并已将 XLV (医疗保健 (Health Care)) 的数据写入到工作表 'XLV'。
获取数据条数: 2515
数据起始日期: 2015-01-02
数据结束日期: 2024-12-30

正在获取 农林牧渔 (Agriculture) (DBA) 的数据...
成功获取并已将 DBA (农林牧渔 (Agriculture)) 的数据写入到工作表 'DBA'。
获取数据条数: 2515
数据起始日期: 2015-01-02
数据结束日期: 2024-12-30

正在获取 采矿业 (Mining) (XME) 的数据...
成功获取并已将 XME (采矿业 (Mining)) 的数据写入到工作表 'XME'。
获取数据条数: 2515
数据起始日期: 2015-01-02
数据结束日期: 2024-12-30

正在获取 制造业 (Industrials/Manufacturing) (XLI) 的数据...
成功获取并已将 XLI (制造业 (Industrials/Manufacturing)) 的数据写入到工作表 'XLI'。
获取数据条数: 2515
数据起始日期: 2015-01-02
数据结束日期: 2024-12-30

正在获取 水电煤气 (Utilities) (XLU) 的数据...
成功获取并已将 XLU (水电煤气 (Utilities)) 的数据写入到工作表 'XLU'。
获取数据条数: 2515
数据起始日期: 2015-01-02
数据结束日期: 2024-12-30

正在获取 房地产 (Real Estate) (XLRE) 的数据...
成功获取并已将 XLRE (房地产 (Real Estate)) 的数据写入到工作表 'XLRE'。
获取数据条数: 2322
数据起始日期: 2015-10-08
数据结束日期: 2024-12-30

正在获取 批发和零售业 (Retail) (XRT) 的数据...
成功获取并已将 XRT (批发和零售业 (Retail)) 的数据写入到工作表 'XRT'。
获取数据条数: 2515
数据起始日期: 2015-01-

# 第一题-第二小题

用短期的、以美元计价的无风险利率--13周美国国债收益率（^IRX）作为无风险利率的代理指标。

In [60]:
# 导入所需库
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

def fetch_risk_free_rate_data(ticker_symbol, start_date_str, end_date_str, excel_writer):
    """
    获取指定代码的日度历史数据作为无风险利率，并将其保存到Excel工作表。
    默认使用Adj Close计算每日无风险利率 (annualized_yield / 100) / 252。

    参数:
    ticker_symbol (str): 无风险利率代理的 ticker 代码 (例如 "^IRX" 代表13周美债收益率)。
    start_date_str (str): 开始日期，格式 "YYYY-MM-DD"。
    end_date_str (str): 结束日期，格式 "YYYY-MM-DD"。
    excel_writer (pd.ExcelWriter): 用于写入Excel文件的写入器对象。

    返回:
    pd.Series: 包含每日无风险利率的Series，索引为日期。若获取失败则返回None。
    """
    print(f"\n正在获取无风险利率数据 ({ticker_symbol})...")
    daily_rf_series = None
    try:
        rf_data = yf.download(ticker_symbol, start=start_date_str, end=end_date_str, progress=False)
        if rf_data.empty:
            print(f"未能获取到 {ticker_symbol} 的数据。")
            empty_df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
            empty_df.to_excel(excel_writer, sheet_name=f"{ticker_symbol}_RF", index_label="Date")
        else:
            rf_data.to_excel(excel_writer, sheet_name=f"{ticker_symbol}_RF_Raw", index_label="Date")
            print(f"成功获取并已将 {ticker_symbol} (无风险利率原始数据) 写入到工作表 '{ticker_symbol}_RF_Raw'。")

            # 使用 'Adj Close' 或 'Close' 计算每日无风险利率
            # Yahoo Finance对^IRX等利率数据提供的通常是年化百分比收益率
            price_col = 'Adj Close' if 'Adj Close' in rf_data.columns else 'Close'
            if price_col not in rf_data.columns:
                print(f"错误: {ticker_symbol} 数据中缺少 '{price_col}' 列来计算无风险利率。")
                return None

            # 将年化收益率转换为每日小数形式 (例如 5% -> 0.05 / 252)
            # 假设一年252个交易日
            daily_rf_series = (rf_data[price_col] / 100) / 252
            daily_rf_series.name = "DailyRiskFreeRate"

            # 将计算出的每日无风险利率也保存到Excel
            daily_rf_df = pd.DataFrame(daily_rf_series)
            daily_rf_df.to_excel(excel_writer, sheet_name=f"{ticker_symbol}_Daily_RF", index_label="Date")
            print(f"已将计算的每日无风险利率 ({ticker_symbol}) 写入到工作表 '{ticker_symbol}_Daily_RF'。")

    except Exception as e:
        print(f"获取无风险利率数据 ({ticker_symbol}) 时发生错误: {e}")
        error_df = pd.DataFrame({'Error': [str(e)]})
        error_df.to_excel(excel_writer, sheet_name=f"{ticker_symbol}_RF_Error", index=False)

    return daily_rf_series

def fetch_etf_data_to_excel(start_date_str, end_date_str, industry_etfs, output_filename="industry_data.xlsx", risk_free_ticker="^IRX"):
    """
    获取指定日期范围内多个ETF的日度历史数据和无风险利率数据。
    将每个ETF的原始数据和无风险利率数据保存到Excel文件的不同工作表中。
    返回所有ETF的调整后收盘价DataFrame、每日回报率字典以及每日无风险利率Series。

    参数:
    start_date_str (str): 开始日期，格式 "YYYY-MM-DD"
    end_date_str (str): 结束日期，格式 "YYYY-MM-DD"
    industry_etfs (dict): 包含行业名称和对应ETF代码的字典。
    output_filename (str): 输出的Excel文件名。
    risk_free_ticker (str): 用于获取无风险利率的ticker符号。

    返回:
    tuple: (all_collected_adj_closes, etf_collected_returns, daily_rf_series)
           - all_collected_adj_closes (pd.DataFrame): 包含所有ETF调整后收盘价的DataFrame。
           - etf_collected_returns (dict): 包含每个ETF每日回报率 (pd.Series) 的字典。
           - daily_rf_series (pd.Series): 包含每日无风险利率的Series。
           若日期格式错误或Excel写入失败，则返回 (None, None, None)。
    """

    try:
        datetime.strptime(start_date_str, "%Y-%m-%d")
        datetime.strptime(end_date_str, "%Y-%m-%d")
    except ValueError:
        print("错误：日期格式不正确。请使用 YYYY-MM-DD 格式。")
        return None, None, None

    all_collected_adj_closes = pd.DataFrame()
    etf_collected_returns = {}
    daily_rf_series = None

    try:
        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
            print(f"开始获取数据并写入到 {output_filename}...")

            # 1. 获取行业ETF数据
            for industry_name, ticker_symbol in industry_etfs.items():
                print(f"\n正在获取 {industry_name} ({ticker_symbol}) 的数据...")
                try:
                    data = yf.download(ticker_symbol, start=start_date_str, end=end_date_str, progress=False)

                    if data.empty:
                        print(f"未能获取到 {ticker_symbol} 的数据。")
                        empty_df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
                        empty_df.to_excel(writer, sheet_name=ticker_symbol, index_label="Date")
                        etf_collected_returns[ticker_symbol] = pd.Series(dtype=float)
                    else:
                        data.to_excel(writer, sheet_name=ticker_symbol, index_label="Date")
                        print(f"成功获取并已将 {ticker_symbol} ({industry_name}) 的原始数据写入到工作表 '{ticker_symbol}'。")

                        price_column_for_returns = 'Adj Close' if 'Adj Close' in data.columns else 'Close'
                        price_series = data[price_column_for_returns].copy()

                        daily_returns = price_series.pct_change().dropna()
                        etf_collected_returns[ticker_symbol] = daily_returns
                        all_collected_adj_closes[ticker_symbol] = price_series

                except Exception as e:
                    print(f"获取 {ticker_symbol} ({industry_name}) 数据时发生错误: {e}")
                    error_df = pd.DataFrame({'Error': [str(e)]})
                    error_df.to_excel(writer, sheet_name=f"{ticker_symbol}_Error", index=False)
                    etf_collected_returns[ticker_symbol] = pd.Series(dtype=float)

            # 2. 获取无风险利率数据
            if risk_free_ticker:
                daily_rf_series = fetch_risk_free_rate_data(risk_free_ticker, start_date_str, end_date_str, writer)

            print(f"\n所有数据处理完毕。Excel文件 '{output_filename}' 已成功保存。")

        return all_collected_adj_closes, etf_collected_returns, daily_rf_series

    except Exception as e:
        print(f"创建或写入Excel文件 '{output_filename}' 时发生严重错误: {e}")
        return None, None, None

if __name__ == '__main__':
    print("数据获取模块 (etf_data_downloader.py)")
    print("该模块包含 fetch_etf_data_to_excel 和 fetch_risk_free_rate_data 函数。")

    # 示例用法:
    test_industry_etfs = {"科技 (Technology)": "XLK"}
    test_start_date = "2015-01-01"
    test_end_date = "2024-12-31"
    test_output_file = "test_etf_and_rf_data.xlsx"
    print(f"\n运行测试下载，数据将保存到 {test_output_file}...")
    adj_closes, returns, rf_rate = fetch_etf_data_to_excel(
        test_start_date,
        test_end_date,
        test_industry_etfs,
        test_output_file,
        risk_free_ticker="^IRX" # 指定获取无风险利率
    )
    if adj_closes is not None and returns is not None:
       print("\n测试ETF数据获取成功。")
       if rf_rate is not None and not rf_rate.empty:
           print("测试无风险利率数据获取成功。")
           print(f"平均每日无风险利率: {rf_rate}")
       elif rf_rate is not None and rf_rate.empty:
           print("无风险利率数据为空，可能日期范围内无数据。")
       else:
           print("测试无风险利率数据获取失败。")
    else:
       print("\n测试数据获取失败。")


数据获取模块 (etf_data_downloader.py)
该模块包含 fetch_etf_data_to_excel 和 fetch_risk_free_rate_data 函数。

运行测试下载，数据将保存到 test_etf_and_rf_data.xlsx...
开始获取数据并写入到 test_etf_and_rf_data.xlsx...

正在获取 科技 (Technology) (XLK) 的数据...
成功获取并已将 XLK (科技 (Technology)) 的原始数据写入到工作表 'XLK'。

正在获取无风险利率数据 (^IRX)...
成功获取并已将 ^IRX (无风险利率原始数据) 写入到工作表 '^IRX_RF_Raw'。
已将计算的每日无风险利率 (^IRX) 写入到工作表 '^IRX_Daily_RF'。

所有数据处理完毕。Excel文件 'test_etf_and_rf_data.xlsx' 已成功保存。

测试ETF数据获取成功。
测试无风险利率数据获取成功。
平均每日无风险利率: Ticker              ^IRX
Date                    
2015-01-02  5.952381e-07
2015-01-05  1.190476e-07
2015-01-06  7.936508e-07
2015-01-07  7.936508e-07
2015-01-08  7.142857e-07
...                  ...
2024-12-23  1.672619e-04
2024-12-24  1.666667e-04
2024-12-26  1.672619e-04
2024-12-27  1.657936e-04
2024-12-30  1.659524e-04

[2514 rows x 1 columns]


# 第一题-第三小题

## **假设陈述**：

- 交易日数量（252 天）

- 假设投资者的效用函数 (Utility Function) 为：
U=E(R)-1/2Aσ^2，
风险厌恶系数 A（设定为 5.0）

- 使用历史平均值作为预期回报

- 允许卖空

##**数据准备**

将交易型开放式指数基金（ETF）的每日回报率与每日无风险利率序列对齐，以确保计算数据的一致性。

# 第一题-第四小题

## 背景知识


我们使用投资组合理论中经典的“三步法”来推导最优风险资产组合和最优完整投资组合的结构。这“三步”指的是：

1.  **确定风险资产的有效边界 (Efficient Frontier of Risky Assets)**：找出由所有可用风险资产构成的、在给定风险水平下提供最高预期回报，或在给定预期回报水平下风险最低的投资组合集合。
2.  **确定最优风险资产组合 (Optimal Risky Portfolio)**：在风险资产的有效边界上，找到夏普比率 (Sharpe Ratio) 最大的那个投资组合。这个组合也被称为切点投资组合 (Tangency Portfolio)。
3.  **确定最优完整投资组合 (Optimal Complete Portfolio)**：将最优风险资产组合与无风险资产相结合，根据投资者的风险厌恶程度来决定在两者之间如何分配资金。

下面我们将详细阐述每个步骤的量化过程。

---

### 第一步：确定风险资产的有效边界

假设市场上有 $N$ 种风险资产。我们需要估计这些资产的以下参数：

* 预期收益率向量 (Expected Returns): $E(\mathbf{R}) = [E(R_1), E(R_2), ..., E(R_N)]^T$
* 协方差矩阵 (Covariance Matrix): $\mathbf{\Sigma}$，这是一个 $N \times N$ 的矩阵，其中元素 $\sigma_{ij}$ 表示资产 $i$ 和资产 $j$ 收益率的协方差，对角线元素 $\sigma_{ii} = \sigma_i^2$ 表示资产 $i$ 收益率的方差。

一个由这 $N$ 种风险资产构成的投资组合 $P$ 的权重向量为 $\mathbf{w} = [w_1, w_2, ..., w_N]^T$，其中 $\sum_{i=1}^{N} w_i = 1$。

该投资组合 $P$ 的预期收益率 $E(R_P)$ 和方差 $\sigma_P^2$ 分别为：

$$E(R_P) = \mathbf{w}^T E(\mathbf{R}) = \sum_{i=1}^{N} w_i E(R_i)$$
$$\sigma_P^2 = \mathbf{w}^T \mathbf{\Sigma} \mathbf{w} = \sum_{i=1}^{N} \sum_{j=1}^{N} w_i w_j \sigma_{ij}$$

**目标**：对于任意给定的预期收益率水平 $E_0$，我们希望找到使得投资组合方差 $\sigma_P^2$ 最小化的权重向量 $\mathbf{w}$。这是一个约束优化问题：

$$\min_{\mathbf{w}} \quad \frac{1}{2} \mathbf{w}^T \mathbf{\Sigma} \mathbf{w}$$
约束条件：
1.  $\mathbf{w}^T E(\mathbf{R}) = E_0$ (预期收益率等于给定值)
2.  $\mathbf{w}^T \mathbf{1} = 1$ (权重之和为1，其中 $\mathbf{1}$ 是一个元素全为1的 $N \times 1$ 向量)

我们可以使用拉格朗日乘数法来解决这个问题。构造拉格朗日函数：

$$\mathcal{L}(\mathbf{w}, \lambda_1, \lambda_2) = \frac{1}{2} \mathbf{w}^T \mathbf{\Sigma} \mathbf{w} - \lambda_1 (\mathbf{w}^T E(\mathbf{R}) - E_0) - \lambda_2 (\mathbf{w}^T \mathbf{1} - 1)$$

对 $\mathbf{w}$ 求偏导并令其为零：
$$\frac{\partial \mathcal{L}}{\partial \mathbf{w}} = \mathbf{\Sigma} \mathbf{w} - \lambda_1 E(\mathbf{R}) - \lambda_2 \mathbf{1} = \mathbf{0}$$
由此可得：
$$\mathbf{w}^* = \mathbf{\Sigma}^{-1} (\lambda_1 E(\mathbf{R}) + \lambda_2 \mathbf{1})$$

将 $\mathbf{w}^*$ 代入两个约束条件，可以解出 $\lambda_1$ 和 $\lambda_2$。这个过程涉及到求解一个线性方程组。

通过改变 $E_0$ 的值，我们可以得到一系列的最小方差投资组合。这些组合在预期收益率-标准差平面上构成了**最小方差边界 (Minimum Variance Frontier)**。最小方差边界的上半部分，即从**全局最小方差组合 (Global Minimum Variance Portfolio, GMVP)** 开始向右上方延伸的部分，被称为**有效边界 (Efficient Frontier)**。理性的、风险厌恶的投资者只会选择有效边界上的投资组合。

**全局最小方差组合 (GMVP)** 是指在所有可能的风险资产组合中，方差最小的那个组合。其权重可以通过以下优化问题得到：
$$\min_{\mathbf{w}} \quad \frac{1}{2} \mathbf{w}^T \mathbf{\Sigma} \mathbf{w}$$
约束条件：
$$\mathbf{w}^T \mathbf{1} = 1$$
其解为：
$$\mathbf{w}_{GMVP} = \frac{\mathbf{\Sigma}^{-1} \mathbf{1}}{\mathbf{1}^T \mathbf{\Sigma}^{-1} \mathbf{1}}$$

---

### 第二步：确定最优风险资产组合 (P\*)

现在我们引入无风险资产，其收益率为 $R_f$，标准差为 $0$。投资者可以将资金分配在无风险资产和由风险资产构成的某个投资组合 $P$ 之间。这样形成的新的投资组合 $C$ 的预期收益率 $E(R_C)$ 和标准差 $\sigma_C$ 分别为：

$$E(R_C) = (1-w_P) R_f + w_P E(R_P)$$
$$\sigma_C = w_P \sigma_P$$
其中 $w_P$ 是投资于风险组合 $P$ 的权重，$1-w_P$ 是投资于无风险资产的权重。

由此可得，连接无风险资产和任意风险组合 $P$ 的直线被称为**资本配置线 (Capital Allocation Line, CAL)**。其斜率是组合 $P$ 的**夏普比率 (Sharpe Ratio)**：

$$S_P = \frac{E(R_P) - R_f}{\sigma_P}$$

夏普比率衡量了投资组合每承担一单位总风险所能获得的超额收益（相对于无风险利率）。理性的投资者会选择使得夏普比率最大的风险资产组合。这个组合就是**最优风险资产组合 (Optimal Risky Portfolio, P\*)**，也称为**切点投资组合 (Tangency Portfolio)**，因为它位于与风险资产有效边界相切的那条资本配置线的切点上。

**目标**：找到使得夏普比率 $S_P$ 最大的风险资产组合 $P^*$。即：

$$\max_{\mathbf{w}} \quad \frac{\mathbf{w}^T E(\mathbf{R}) - R_f}{(\mathbf{w}^T \mathbf{\Sigma} \mathbf{w})^{1/2}}$$
约束条件：
$$\mathbf{w}^T \mathbf{1} = 1$$

这个优化问题的解，即最优风险资产组合 $P^*$ 的权重向量 $\mathbf{w}^*$ 为：

$$\mathbf{w}^* = \frac{\mathbf{\Sigma}^{-1} (E(\mathbf{R}) - R_f \mathbf{1})}{\mathbf{1}^T \mathbf{\Sigma}^{-1} (E(\mathbf{R}) - R_f \mathbf{1})}$$
其中 $E(\mathbf{R}) - R_f \mathbf{1}$ 是一个 $N \times 1$ 的向量，表示各风险资产相对于无风险利率的预期超额收益。

一旦确定了 $\mathbf{w}^*$，我们就可以计算出最优风险资产组合 $P^*$ 的预期收益率 $E(R_{P^*})$ 和标准差 $\sigma_{P^*}$：

$$E(R_{P^*}) = (\mathbf{w}^*)^T E(\mathbf{R})$$
$$\sigma_{P^*}^2 = (\mathbf{w}^*)^T \mathbf{\Sigma} \mathbf{w}^*$$

这条通过 $(0, R_f)$ 点并与有效边界相切于 $P^*$ 的资本配置线被称为**资本市场线 (Capital Market Line, CML)**，它是所有可能的资本配置线中斜率最大的一条。CML 的方程为：

$$E(R_C) = R_f + \frac{E(R_{P^*}) - R_f}{\sigma_{P^*}} \sigma_C$$

---

### 第三步：确定最优完整投资组合 (O)

现在投资者已经找到了最优风险资产组合 $P^*$。接下来的任务是决定如何在 $P^*$ 和无风险资产之间分配资金，以构建其**最优完整投资组合 (Optimal Complete Portfolio, O)**。这个决策取决于投资者的个人**风险厌恶程度 (Risk Aversion)**。

假设投资者的效用函数 (Utility Function) 为：
$$U = E(R_C) - \frac{1}{2} A \sigma_C^2$$
其中 $A$ 是投资者的风险厌恶系数。$A > 0$ 表示投资者是风险厌恶的，$A$ 值越大，风险厌恶程度越高。

我们将 $E(R_C) = (1-y) R_f + y E(R_{P^*})$ 和 $\sigma_C = y \sigma_{P^*}$ 代入效用函数，其中 $y$ 是投资于最优风险资产组合 $P^*$ 的比例：

$$U(y) = (1-y) R_f + y E(R_{P^*}) - \frac{1}{2} A (y \sigma_{P^*})^2$$
$$U(y) = R_f + y (E(R_{P^*}) - R_f) - \frac{1}{2} A y^2 \sigma_{P^*}^2$$

**目标**：最大化投资者的效用函数 $U(y)$。对 $y$ 求一阶导数并令其为零：

$$\frac{dU}{dy} = E(R_{P^*}) - R_f - A y \sigma_{P^*}^2 = 0$$

解出最优的 $y$ 值，记为 $y^*$：

$$y^* = \frac{E(R_{P^*}) - R_f}{A \sigma_{P^*}^2}$$

这个 $y^*$ 就是投资者应该投资于最优风险资产组合 $P^*$ 的比例。剩余的 $1-y^*$ 的比例将投资于无风险资产。

* 如果 $y^* > 1$，表示投资者借入无风险资金（杠杆投资）来更多地投资于 $P^*$。
* 如果 $0 < y^* \le 1$，表示投资者将一部分资金投资于 $P^*$，一部分投资于无风险资产。
* 如果 $y^* \le 0$，对于高度风险厌恶的投资者，理论上可能出现这种情况，但通常我们假设投资者至少会考虑风险资产。

一旦确定了 $y^*$，最优完整投资组合 $O$ 的预期收益率 $E(R_O)$ 和标准差 $\sigma_O$ 就可以计算出来：

$$E(R_O) = (1-y^*) R_f + y^* E(R_{P^*})$$
$$\sigma_O = y^* \sigma_{P^*}$$

这个最优完整投资组合 $O$ 位于资本市场线 (CML) 上，并且是投资者个人效用曲线（无差异曲线）与 CML 的切点。


## 代码

In [63]:
# 导入所需库
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize # Added for efficient frontier calculation

po = None # 设置为None，以便后续检查

def perform_statistical_analysis(industry_etfs_config, all_adj_closes_data, etf_returns_data_dict, daily_rf_series):
    """
    对提取的ETF数据和无风险利率数据进行统计分析并打印结果。

    参数:
    industry_etfs_config (dict): 行业ETF配置字典。
    all_adj_closes_data (pd.DataFrame): 包含所有ETF调整后收盘价（或收盘价）的DataFrame。
    etf_returns_data_dict (dict): 包含每个ETF每日回报率 (pd.Series) 的字典。
    daily_rf_series (pd.Series): 包含每日无风险利率的Series。可能为None或空。
    """
    print("\n--- 数据分析结果 ---")

    # 1. 计算ETF的平均每日回报率和标准差
    print("\n1. ETF 平均每日回报率和标准差:")
    analysis_results_list = []

    if not etf_returns_data_dict:
        print("  没有可供分析的ETF回报率数据。")
    else:
        for ticker_symbol, returns_series in etf_returns_data_dict.items():
            industry_display_name = "未知行业"
            # 确保 industry_etfs_config 中的值是股票代码
            for name, tick_list in industry_etfs_config.items(): # 假设值可能是列表或单个代码
                # 处理 tick_list 可能是单个字符串或列表的情况
                ticks_to_check = [tick_list] if isinstance(tick_list, str) else tick_list
                if ticker_symbol in ticks_to_check:
                    industry_display_name = name
                    break

            if returns_series is not None and not returns_series.empty:
                avg_return = returns_series.mean()
                std_dev = returns_series.std()
                analysis_results_list.append({
                    "ETF": ticker_symbol,
                    "行业": industry_display_name,
                    "平均每日回报率": avg_return,
                    "每日回报率标准差": std_dev,
                    "年化平均回报率 (估算)": avg_return * 252,
                    "年化标准差 (估算)": std_dev * np.sqrt(252)
                })
            else:
                analysis_results_list.append({
                    "ETF": ticker_symbol,
                    "行业": industry_display_name,
                    "平均每日回报率": np.nan, "每日回报率标准差": np.nan,
                    "年化平均回报率 (估算)": np.nan, "年化标准差 (估算)": np.nan
                })
                print(f"  {ticker_symbol} ({industry_display_name}): 无有效回报率数据进行分析。")

    if analysis_results_list:
        results_summary_df = pd.DataFrame(analysis_results_list)
        # 根据ETF代码排序，以便与优化结果对齐
        if "ETF" in results_summary_df.columns:
             results_summary_df = results_summary_df.sort_values(by="ETF").reset_index(drop=True)
        print(results_summary_df.to_string(index=False))
    elif etf_returns_data_dict:
        print("  所有ETF均无有效回报率数据进行分析。")

    # 2. 计算ETF回报率的相关性矩阵
    print("\n2. ETF 每日回报率相关性矩阵:")
    if all_adj_closes_data is None or all_adj_closes_data.empty or all_adj_closes_data.shape[1] < 2:
        print("  未能收集到足够ETF的价格数据来计算相关性矩阵。")
    else:
        returns_df_for_correlation = all_adj_closes_data.pct_change().dropna()

        if returns_df_for_correlation.shape[1] < 2 :
             print("  计算回报率后，有效ETF数量不足2个，无法计算相关性矩阵。")
        elif returns_df_for_correlation.empty:
            print("  计算得到的回报率数据为空，无法计算相关性矩阵。")
        else:
            correlation_matrix = returns_df_for_correlation.corr()
            print("ETF每日回报率相关性矩阵:")
            pd.set_option('display.width', None); pd.set_option('display.max_columns', None)
            print(correlation_matrix.to_string(float_format="%.3f"))
            pd.reset_option('display.width'); pd.reset_option('display.max_columns')

    # 3. 无风险利率信息
    print("\n3. 无风险利率数据 (例如 ^IRX):")
    annualized_rf_rate_for_optimizer = np.nan # 初始化用于优化器的年化无风险利率
    if daily_rf_series is not None and not daily_rf_series.empty:
        print(f"  已成功获取每日无风险利率数据。")
        print(f"  数据点数量: {len(daily_rf_series)}")

        mean_val_raw = daily_rf_series.mean()
        scalar_mean_daily_rf = np.nan

        if isinstance(mean_val_raw, pd.Series):
            if not mean_val_raw.empty:
                scalar_mean_daily_rf = float(mean_val_raw.iloc[0])
        elif pd.api.types.is_scalar(mean_val_raw):
            scalar_mean_daily_rf = float(mean_val_raw)

        if pd.isna(scalar_mean_daily_rf):
            print(f"  平均每日无风险利率: NaN")
            print(f"  平均年化无风险利率 (估算): NaN")
        else:
            print(f"  平均每日无风险利率: {scalar_mean_daily_rf:.6f} (即 {(scalar_mean_daily_rf * 100):.4f}%)")
            annualized_mean_rf = scalar_mean_daily_rf * 252
            annualized_rf_rate_for_optimizer = annualized_mean_rf # 用于后续优化
            print(f"  平均年化无风险利率 (估算): {annualized_mean_rf:.6f} (即 {(annualized_mean_rf * 100):.4f}%)")

    elif daily_rf_series is not None and daily_rf_series.empty:
        print("  已尝试获取无风险利率数据，但返回为空系列 (可能日期范围内无数据或代码问题)。")
    else:
        print("  未能获取无风险利率数据或获取过程中发生错误。")

    return annualized_rf_rate_for_optimizer

# --- Helper functions for portfolio optimization (assuming these would be in portfolio_optimizer.py or defined globally) ---
def calculate_daily_returns(adj_closes_df):
    """Calculates daily returns from adjusted close prices."""
    if adj_closes_df is None or adj_closes_df.empty:
        return pd.DataFrame()
    return adj_closes_df.pct_change().dropna()

def calculate_annualized_stats(daily_returns_df):
    """Calculates annualized mean returns and covariance matrix."""
    if daily_returns_df is None or daily_returns_df.empty:
        return None, None
    annualized_mean_returns = daily_returns_df.mean() * 252
    annualized_cov_matrix = daily_returns_df.cov() * 252
    return annualized_mean_returns, annualized_cov_matrix

def find_optimal_risky_portfolio(mean_returns, cov_matrix, risk_free_rate):
    """
    Finds the Optimal Risky Portfolio (ORP) - portfolio with the highest Sharpe ratio.
    This is also known as the tangency portfolio.
    """
    num_assets = len(mean_returns)

    # Objective function: minimize negative Sharpe ratio (which is maximizing Sharpe ratio)
    def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        if portfolio_std_dev == 0: # Avoid division by zero
            return np.inf if (portfolio_return - risk_free_rate) < 0 else -np.inf
        sharpe = (portfolio_return - risk_free_rate) / portfolio_std_dev
        return -sharpe

    # Constraints: sum of weights is 1
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    # Bounds for weights (0 to 1, i.e., long only, no short selling)
    bounds = tuple((0, 1) for asset in range(num_assets))

    # Initial guess for weights (equal distribution)
    initial_weights = np.array(num_assets * [1. / num_assets])

    # Optimization
    try:
        result = minimize(neg_sharpe_ratio, initial_weights,
                          args=(mean_returns, cov_matrix, risk_free_rate),
                          method='SLSQP', bounds=bounds, constraints=constraints)
    except Exception as e:
        print(f"  Error during ORP optimization: {e}")
        return None, np.nan, np.nan, np.nan

    if not result.success:
        print(f"  ORP optimization failed: {result.message}")
        return None, np.nan, np.nan, np.nan

    optimal_weights = result.x
    orp_return = np.sum(mean_returns * optimal_weights)
    orp_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    orp_sharpe = (orp_return - risk_free_rate) / orp_volatility

    return optimal_weights, orp_return, orp_volatility, orp_sharpe

def calculate_optimal_complete_portfolio_allocation(orp_expected_return, orp_volatility, risk_free_rate, risk_aversion_coefficient):
    """Calculates the allocation to the ORP in the complete portfolio."""
    if orp_volatility == 0: # Avoid division by zero if ORP has no risk (e.g. single asset = risk-free)
        return 0.0 if orp_expected_return <= risk_free_rate else np.nan # or handle as per specific logic

    y_star = (orp_expected_return - risk_free_rate) / (risk_aversion_coefficient * orp_volatility**2)
    return y_star

# --- NEW: Functions for Efficient Frontier Calculation ---
def portfolio_volatility(weights, mean_returns, cov_matrix):
    """Calculates portfolio volatility (annualized)."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def portfolio_return(weights, mean_returns):
    """Calculates portfolio expected return (annualized)."""
    return np.sum(mean_returns * weights)

def minimize_volatility_objective(weights, mean_returns, cov_matrix):
    """Objective function for scipy.optimize.minimize: portfolio variance."""
    # We minimize variance, which is equivalent to minimizing volatility
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def calculate_and_display_efficient_frontier(mean_returns, cov_matrix, asset_names, risk_free_rate, num_portfolios=50):
    """
    Calculates and displays points on the efficient frontier.
    """
    print("\n--- 步骤 1.5: 计算和展示风险资产的有效边界 ---")
    num_assets = len(mean_returns)
    if num_assets < 2:
        print("  至少需要两个资产才能计算有效边界。")
        return

    results_array = [] # To store volatility, return, sharpe, and weights

    # Define target returns for the frontier
    # We'll span from slightly above min individual return to slightly below max individual return
    # or a wider range if assets have very similar returns.
    min_ind_return = mean_returns.min()
    max_ind_return = mean_returns.max()

    # If all returns are very close, linspace might create identical target returns.
    # Ensure a reasonable spread for target_returns.
    if np.isclose(min_ind_return, max_ind_return):
        # If all assets have almost the same return, the "frontier" is less meaningful in terms of return spread.
        # We can generate a few points around this mean return with slight variations.
        target_returns = np.linspace(min_ind_return - 0.01, max_ind_return + 0.01, num_portfolios)
    else:
        target_returns = np.linspace(min_ind_return, max_ind_return, num_portfolios)

    initial_weights = np.array([1./num_assets] * num_assets)
    bounds = tuple((0, 1) for asset in range(num_assets)) # Long-only

    for target_ret in target_returns:
        # Constraints for the optimization:
        # 1. Sum of weights must be 1
        # 2. Portfolio return must equal the target_ret for this point on the frontier
        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
            {'type': 'eq', 'fun': lambda weights: portfolio_return(weights, mean_returns) - target_ret}
        )

        # Optimization to find weights that minimize volatility for the target_ret
        # We pass mean_returns and cov_matrix to the objective function via 'args'
        optimizer_result = minimize(minimize_volatility_objective,
                                    initial_weights,
                                    args=(mean_returns, cov_matrix), # Pass additional args to objective
                                    method='SLSQP',
                                    bounds=bounds,
                                    constraints=constraints)

        if optimizer_result.success:
            optimized_weights = optimizer_result.x
            frontier_vol = portfolio_volatility(optimized_weights, mean_returns, cov_matrix)
            frontier_ret = portfolio_return(optimized_weights, mean_returns) # Should be very close to target_ret
            frontier_sharpe = (frontier_ret - risk_free_rate) / frontier_vol if frontier_vol > 1e-6 else np.nan

            # Store results including weights
            result_entry = [frontier_vol, frontier_ret, frontier_sharpe]
            result_entry.extend(optimized_weights)
            results_array.append(result_entry)
        # else:
            # print(f"  Optimization failed for target return: {target_ret:.4f} - {optimizer_result.message}")


    if not results_array:
        print("  未能计算有效边界上的投资组合。")
        return

    # Create a DataFrame for better display
    columns = ['Volatility (σ)', 'Return E(R)', 'Sharpe Ratio'] + [f'W_{name}' for name in asset_names]
    frontier_df = pd.DataFrame(results_array, columns=columns)

    # Sort by volatility to ensure points are plotted in order if graphing later
    frontier_df = frontier_df.sort_values(by='Volatility (σ)').reset_index(drop=True)

    print("\n有效边界上的投资组合点 (部分示例):")
    # Display a subset or summary if too many points
    display_df = frontier_df
    if len(frontier_df) > 20: # Show head and tail if many points
        print("  (显示前10和后10个点)")
        display_df = pd.concat([frontier_df.head(10), frontier_df.tail(10)])

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        # Format numbers for better readability
        formatted_df = display_df.copy()
        formatted_df['Volatility (σ)'] = formatted_df['Volatility (σ)'].map('{:.4%}'.format)
        formatted_df['Return E(R)'] = formatted_df['Return E(R)'].map('{:.4%}'.format)
        formatted_df['Sharpe Ratio'] = formatted_df['Sharpe Ratio'].map('{:.4f}'.format)
        for col in [f'W_{name}' for name in asset_names]:
            if col in formatted_df:
                 formatted_df[col] = formatted_df[col].map('{:.4%}'.format)
        print(formatted_df.to_string(index=False))

    print("\n  注意: 有效边界代表在给定预期回报水平下风险最低的投资组合集合，")
    print("  或者在给定风险水平下预期回报最高的投资组合集合。")
    print("  可以使用这些点来绘制有效边界图 (例如使用 matplotlib)。")
    # Example for plotting (requires matplotlib):
    # import matplotlib.pyplot as plt
    # plt.figure(figsize=(10, 6))
    # plt.plot(frontier_df['Volatility (σ)'], frontier_df['Return E(R)'], 'o-', label='Efficient Frontier')
    # plt.title('Efficient Frontier of Risky Assets')
    # plt.xlabel('Annualized Volatility (Standard Deviation)')
    # plt.ylabel('Annualized Expected Return')
    # plt.legend()
    # plt.grid(True)
    # plt.show()


# --- 主程序 ---
if __name__ == "__main__":
    # --- 配置参数 ---
    industry_etfs_config = {
    "医疗保健 (Health Care)": "XLV",
    "农林牧渔 (Agriculture)": "DBA",  # Invesco DB Agriculture Fund
    "采矿业 (Mining)": "XME",  # SPDR S&P Metals & Mining ETF
    "制造业 (Industrials/Manufacturing)": "XLI",  # Industrial Select Sector SPDR Fund (覆盖广泛的制造业和工业)
    "水电煤气 (Utilities)": "XLU",  # Utilities Select Sector SPDR Fund
    "房地产 (Real Estate)": "XLRE",
    "批发和零售业 (Retail)": "XRT",  # SPDR S&P Retail ETF
    "交通运输 (Transportation)": "IYT",  # iShares U.S. Transportation ETF
    "住宿和餐饮业 (Consumer Discretionary/Travel)": "PEJ", # Invesco Leisure and Entertainment ETF (更侧重休闲娱乐，间接包含部分)
    "金融业 (Financials)": "XLF"  # Financial Select Sector SPDR Fund
    }

    etf_tickers_for_optimization = list(industry_etfs_config.values())

    start_date_input = "2015-01-01"
    current_year = datetime.now().year
    requested_end_year = 2024
    if current_year < requested_end_year:
        print(f"提示: 当前年份 ({current_year}) 早于请求的结束年份 ({requested_end_year})，数据将获取至当前最新。")
    end_date_input = f"{requested_end_year}-12-31"
    excel_file_name = f"行业ETF与无风险利率历史数据_{start_date_input.split('-')[0]}_至_{end_date_input.split('-')[0]}.xlsx"

    risk_free_rate_ticker_symbol = "^IRX"

    # --- 数据获取 ---
    fetched_adj_closes, fetched_returns_dict, fetched_daily_rf_series = None, None, None

    # --- 实际数据获取代码 (取消注释以使用) ---
    # This part is assumed to be provided by the user as 'etf_data_downloader.py'
    # For demonstration, we will mock this data if the import fails.
    MOCK_DATA_MODE = False # Set to True to use mock data if etf_data_downloader is not available
    try:
        # from etf_data_downloader import fetch_etf_data_to_excel
        print("尝试从 etf_data_downloader 模块导入 fetch_etf_data_to_excel 函数。")

        fetched_adj_closes, fetched_returns_dict, fetched_daily_rf_series = fetch_etf_data_to_excel(
             start_date_str=start_date_input,
             end_date_str=end_date_input,
             industry_etfs=industry_etfs_config,
             output_filename=excel_file_name,
             risk_free_ticker=risk_free_rate_ticker_symbol
        )
        if fetched_adj_closes is not None:
             print(f"成功获取了 {len(fetched_adj_closes.columns)} 个ETF的调整后收盘价数据。")
             etf_tickers_for_optimization = fetched_adj_closes.columns.tolist()
        else:
            MOCK_DATA_MODE = True # Fallback to mock data if fetching returns None
            print("获取数据返回None，将尝试使用模拟数据（如果启用）。")


    except ImportError:
        print("错误: 未能从 etf_data_downloader.py 导入 fetch_etf_data_to_excel。")
        print("请确保将您的数据下载脚本保存为 etf_data_downloader.py 文件，并与此脚本放在同一目录下。")
        MOCK_DATA_MODE = True
    except Exception as e:
        print(f"调用 fetch_etf_data_to_excel 时发生未知错误: {e}")
        MOCK_DATA_MODE = True

    if MOCK_DATA_MODE:
        print("\n--- 使用模拟数据进行演示 ---")
        # Create mock data if etf_data_downloader.py is not available or fails
        num_days = 252 * 5 # 5 years of data
        mock_dates = pd.date_range(start=start_date_input, periods=num_days, freq='B')
        mock_data = {}
        np.random.seed(42) # For reproducibility
        for ticker in etf_tickers_for_optimization:
            # Simulate some price movements
            price_path = 100 + np.random.randn(num_days).cumsum() * 0.1 + np.sin(np.linspace(0, 20, num_days)) * 5
            mock_data[ticker] = np.maximum(1, price_path) # Ensure prices are positive

        fetched_adj_closes = pd.DataFrame(mock_data, index=mock_dates)

        fetched_returns_dict = {}
        for ticker in fetched_adj_closes.columns:
            fetched_returns_dict[ticker] = fetched_adj_closes[ticker].pct_change().dropna()

        # Mock risk-free rate (e.g., constant 1% annualized)
        mock_rf_daily = (1 + 0.01)**(1/252) - 1
        fetched_daily_rf_series = pd.Series([mock_rf_daily] * num_days, index=mock_dates)
        print(f"已生成 {len(fetched_adj_closes.columns)} 个ETF的模拟调整后收盘价数据。")
        print(f"已生成模拟无风险利率数据。")


    # --- 初步统计分析 ---
    annualized_rf_from_stats = np.nan
    if fetched_adj_closes is not None and fetched_returns_dict is not None:
        if not fetched_adj_closes.empty:
            etf_tickers_for_optimization = fetched_adj_closes.columns.tolist()
        else:
            print("警告: 获取的调整后收盘价数据为空，无法确定用于优化的ETF代码。")
            etf_tickers_for_optimization = []

        annualized_rf_from_stats = perform_statistical_analysis(
            industry_etfs_config=industry_etfs_config,
            all_adj_closes_data=fetched_adj_closes,
            etf_returns_data_dict=fetched_returns_dict,
            daily_rf_series=fetched_daily_rf_series
        )
    else:
        print("\n由于数据获取失败或未完成，无法进行初步统计分析。")

    # --- 投资组合优化 ---
    if fetched_adj_closes is None or fetched_adj_closes.empty:
        print("\n调整后收盘价数据为空，无法进行投资组合优化。")
    elif not etf_tickers_for_optimization:
        print("\n没有有效的ETF代码用于优化。")
    else:
        print("\n--- 投资组合优化分析 (基于 portfolio_optimizer 模块功能) ---")

        current_etf_universe_for_opt = fetched_adj_closes.columns.tolist()
        if not current_etf_universe_for_opt:
            print("错误: 调整后收盘价数据中没有可用的ETF列进行优化。")
        elif len(current_etf_universe_for_opt) < 2 :
            print(f"警告: 只有 {len(current_etf_universe_for_opt)} 个ETF ({current_etf_universe_for_opt}) 可供优化。至少需要2个ETF进行有意义的组合优化。")
            if len(current_etf_universe_for_opt) == 1:
                 print("  单个ETF的“最优组合”即为其自身100%权重。")

        print("\n--- 步骤一：计算风险-收益特征 ---") # Title slightly adjusted
        daily_returns_for_opt_df = calculate_daily_returns(fetched_adj_closes[current_etf_universe_for_opt])

        if daily_returns_for_opt_df.empty:
            print("未能从调整后收盘价计算每日收益率 (用于优化)，跳过优化。")
        else:
            annualized_mean_returns, annualized_cov_matrix = calculate_annualized_stats(daily_returns_for_opt_df)

            if annualized_mean_returns is None or annualized_cov_matrix is None:
                print("未能计算年化统计数据 (用于优化)，跳过优化。")
            else:
                print("\n优化模块计算的年化统计数据:")
                print("年化预期收益率 E(R):")
                print(annualized_mean_returns.loc[current_etf_universe_for_opt].to_string(float_format="%.4f"))
                print("\n年化协方差矩阵 Σ:")
                print(annualized_cov_matrix.loc[current_etf_universe_for_opt, current_etf_universe_for_opt].to_string(float_format="%.6f"))

                optimizer_rf_rate = annualized_rf_from_stats
                if pd.isna(optimizer_rf_rate):
                    print("\n警告: 未能从数据中获取有效的年化无风险利率。")
                    optimizer_rf_rate = 0.01
                    print(f"将使用默认年化无风险利率进行优化: {optimizer_rf_rate:.2%}")
                else:
                    print(f"\n将使用从数据计算得到的年化无风险利率进行优化: {optimizer_rf_rate:.4%}")

                # --- NEW: Call to calculate and display efficient frontier ---
                if len(current_etf_universe_for_opt) >= 2: # Only if enough assets
                    calculate_and_display_efficient_frontier(
                        mean_returns=annualized_mean_returns.loc[current_etf_universe_for_opt],
                        cov_matrix=annualized_cov_matrix.loc[current_etf_universe_for_opt, current_etf_universe_for_opt],
                        asset_names=current_etf_universe_for_opt,
                        risk_free_rate=optimizer_rf_rate,
                        num_portfolios=50 # You can adjust the number of points on the frontier
                    )
                else:
                    print("\n  资产数量不足 (<2)，跳过有效边界计算。")


                print("\n--- 步骤二：寻找最优风险资产组合 (ORP) ---")
                # Ensure enough assets for ORP calculation
                if len(current_etf_universe_for_opt) >= 1: # ORP can be found for 1 asset (trivial) but meaningful for >=2
                    optimal_weights_orp, er_orp, vol_orp, sharpe_orp = find_optimal_risky_portfolio(
                        mean_returns=annualized_mean_returns.loc[current_etf_universe_for_opt],
                        cov_matrix=annualized_cov_matrix.loc[current_etf_universe_for_opt, current_etf_universe_for_opt],
                        risk_free_rate=optimizer_rf_rate
                    )

                    if optimal_weights_orp is not None:
                        print("\n最优风险资产组合 (ORP) 特征:")
                        print(f"  预期年化收益率 E(R_orp): {er_orp:.4%}")
                        print(f"  年化波动率 (标准差) σ_orp: {vol_orp:.4%}")
                        print(f"  夏普比率 Sharpe_orp: {sharpe_orp:.4f}")
                        print("  资产权重:")
                        for i, ticker in enumerate(current_etf_universe_for_opt):
                            print(f"    {ticker}: {optimal_weights_orp[i]:.4%}")

                        print("\n--- 步骤三：确定最优完整组合 ---")
                        RISK_AVERSION_COEFFICIENT = 5.0
                        print(f"假设的投资者风险厌恶系数 (A): {RISK_AVERSION_COEFFICIENT}")

                        y_star = calculate_optimal_complete_portfolio_allocation(
                            orp_expected_return=er_orp,
                            orp_volatility=vol_orp,
                            risk_free_rate=optimizer_rf_rate,
                            risk_aversion_coefficient=RISK_AVERSION_COEFFICIENT
                        )

                        if not np.isnan(y_star):
                            print(f"\n投资于最优风险资产组合 (ORP) 的资金比例 (y*): {y_star:.4%}")
                            print(f"投资于无风险资产的资金比例 (1 - y*): {(1 - y_star):.4%}")

                            er_complete = y_star * er_orp + (1 - y_star) * optimizer_rf_rate
                            vol_complete = abs(y_star) * vol_orp

                            print("\n最优完整组合特征:")
                            print(f"  预期年化收益率 E(R_complete): {er_complete:.4%}")
                            print(f"  年化波动率 (标准差) σ_complete: {vol_complete:.4%}")

                            print("\n最终各ETF在总投资组合中的配置比例 (占总资产的百分比):")
                            for i, ticker in enumerate(current_etf_universe_for_opt):
                                print(f"    {ticker}: {(optimal_weights_orp[i] * y_star):.4%}")
                        else:
                            print("未能计算最优完整组合的配置比例。")
                    else:
                        print("未能找到最优风险资产组合。")
                else:
                    print("  资产数量不足，无法计算最优风险资产组合。")


    print(f"\n您可以打开 '{excel_file_name}' 文件查看下载的原始价格数据 (如果数据获取成功)。")
    print("每个行业ETF的原始价格数据存储在其股票代码命名的工作表中。")
    print(f"无风险利率 ({risk_free_rate_ticker_symbol}) 的原始数据和计算的每日利率也保存在对应工作表中。")
    print("统计分析和投资组合优化结果已打印在控制台输出中。")
    print("\n--- 脚本执行完毕 ---")


尝试从 etf_data_downloader 模块导入 fetch_etf_data_to_excel 函数。
开始获取数据并写入到 行业ETF与无风险利率历史数据_2015_至_2024.xlsx...

正在获取 医疗保健 (Health Care) (XLV) 的数据...
成功获取并已将 XLV (医疗保健 (Health Care)) 的原始数据写入到工作表 'XLV'。

正在获取 农林牧渔 (Agriculture) (DBA) 的数据...
成功获取并已将 DBA (农林牧渔 (Agriculture)) 的原始数据写入到工作表 'DBA'。

正在获取 采矿业 (Mining) (XME) 的数据...
成功获取并已将 XME (采矿业 (Mining)) 的原始数据写入到工作表 'XME'。

正在获取 制造业 (Industrials/Manufacturing) (XLI) 的数据...
成功获取并已将 XLI (制造业 (Industrials/Manufacturing)) 的原始数据写入到工作表 'XLI'。

正在获取 水电煤气 (Utilities) (XLU) 的数据...
成功获取并已将 XLU (水电煤气 (Utilities)) 的原始数据写入到工作表 'XLU'。

正在获取 房地产 (Real Estate) (XLRE) 的数据...
成功获取并已将 XLRE (房地产 (Real Estate)) 的原始数据写入到工作表 'XLRE'。

正在获取 批发和零售业 (Retail) (XRT) 的数据...
成功获取并已将 XRT (批发和零售业 (Retail)) 的原始数据写入到工作表 'XRT'。

正在获取 交通运输 (Transportation) (IYT) 的数据...
成功获取并已将 IYT (交通运输 (Transportation)) 的原始数据写入到工作表 'IYT'。

正在获取 住宿和餐饮业 (Consumer Discretionary/Travel) (PEJ) 的数据...
成功获取并已将 PEJ (住宿和餐饮业 (Consumer Discretionary/Travel)) 的原始数据写入到工作表 'PEJ'。

正在获取 金融业 (Financials) (XLF) 的数据...